##### CODE BELOW ARE COPIED FROM DAY1

In [1]:
# direct sum
function ⨁(𝑨::AbstractArray{𝕋, ℕ}, 𝑩::AbstractArray{𝕋, ℕ}, axes::Tuple) where {𝕋, ℕ}
    𝑪 = []

    s𝑨 = size(𝑨) # store size of 𝑨 and 𝑩 to calculate size of 𝑪
    s𝑩 = size(𝑩)
    s𝑪 = []

    if length(axes) == ℕ
        s𝑪 = [x+y for (x, y) in zip(s𝑨, s𝑩)]
        𝑪 = zeros(s𝑪...)
        𝑩start = CartesianIndex([x+1 for x in s𝑨]...)
        𝑩end = CartesianIndex(s𝑪...)
    else
        if sum([[x!=y for (x, y) in zip(s𝑨, s𝑩)][x] for x in axes]) != 0
            error("parameter wrong")
        end

        # if x in axes, this dim's data will not expand
        #for x in 1:N if x in axes push!(s𝑪, s𝑨[x]) else push!(s𝑪, s𝑨[x]+s𝑩[x]) end end
        [x in axes ? s𝑪 : s𝑨[x]+s𝑩[x] for x in 1:N] # NOTES:P2
        𝑪 = zeros(s𝑪...)

        # construct 𝑩start
        tmp = ones(Int64, N) # NOTES:P1
        for x in 1:N if !(x in axes) tmp[x]=1+s𝑨[x] end end
        𝑩start = CartesianIndex((tmp...,))
        𝑩end = CartesianIndex(s𝑪...)
    end

    𝑨start = CartesianIndices(𝑪)[1]
    𝑨end = CartesianIndex(s𝑨...)
    𝑪[𝑨start:𝑨end] = 𝑨
    𝑪[𝑩start:𝑩end] = 𝑩

    return 𝑪
end

function contract(a::AbstractArray{Ta, Na}, b::AbstractArray{Tb, Nb}, dima::Tuple, dimb::Tuple) where {Ta, Tb, Na, Nb}
    if [size(a)[x] for x in dima] != [size(b)[x] for x in dimb] error("size is wrong") end
    if length(dima) == 0
        a_size = [size(a)...]
        b_size = [size(b)...]
        ra = reshape(a, prod(a_size), 1)
        rb = reshape(b, 1, prod(b_size))
        return reshape(ra*rb, Tuple([a_size; b_size]))
    end

    a_left = filter(x -> !(x in dima), [1:Na...])
    a_right = [dima...]
    a_perm = [a_left; a_right]
    a_reshape_size = [size(a)[x] for x in a_left]
    a_left_len = prod(a_reshape_size)
    a_right_len = prod([size(a)[x] for x in a_right])

    b_left = [dimb...]
    b_right = filter(x -> !(x in dimb), [1:Nb...])
    b_perm = [b_left; b_right]
    b_left_len = prod([size(b)[x] for x in b_left])
    b_reshape_size = [size(b)[x] for x in b_right]
    b_right_len = prod(b_reshape_size)

    am = permutedims(a, a_perm)
    bm = permutedims(b, b_perm)

    # NOTES: I have spent alot of time on these codes
    ra = reshape(am, (a_left_len, a_right_len))
    rb = reshape(bm, (b_left_len, b_right_len))
    
    return Number.(reshape(ra*rb, Tuple([a_reshape_size; b_reshape_size])))
end

# The right version for kron
function ⨂(t1::AbstractArray, t2::AbstractArray)
    if ndims(t1) != ndims(t2)
        error("ndims t1 must be equaled to t2")
    end

    T = promote_type(eltype(t2), eltype(t1))
    ret = zeros(T, size(t1).*size(t2))
    for i in CartesianIndices(t1)
        c_start = CartesianIndex(.+(.*(.-(Tuple(i), 1), size(t2)), 1))
        c_end = CartesianIndex(Tuple(i).*size(t2))
        ret[c_start:c_end] = t1[i] * t2
    end
    return ret
end

⨂ (generic function with 1 method)

##### HOMEWORK
1. single-qubit operation
2. two-qubits operation
3. measurement

In [2]:
#=

Convention for MPS

     i₂
     |
     |
i₁---⊕---i₃

=#
struct MPS
    data::Vector{Array{Number, 3}}
end

#=

Convention for OnebodyGate -- Gate₁

     i₁
     |
     ⊕
     |
     i₂

=#
struct Gate₁
    op::Array{𝕋, 2} where {𝕋<:Complex{Float64}}
end

#=

Convention for TwobodyGate -- Gate₂

     i₁ i₂
     |  |
    ------
    ------
     |  |
     i₃ i₄

=#
struct Gate₂
    i::Array{Int64, 1}
    op::Array{𝕋, 4} where {𝕋<:Complex{Float64}}
end

In [23]:
function initmps(basis::AbstractArray)
    res = []
    for x in basis
        t = []
        if x == 0
            t = reshape([1,0], 1, 2, 1)
        elseif x == 1
            t = reshape([0,1], 1, 2, 1)
        else
            x = rand()
            y = sqrt(1-x)
            t = reshape([sqrt(x), y], 1, 2, 1)
        end
        push!(res, Number.(t))
    end
    return MPS(res)
end

function __apply_single_qb_gate(sys::MPS, i::Int64, gate::AbstractArray)
    sys.data[i] = permutedims(contract(sys.data[i], gate, (2,), (2,)), [1,3,2])
end

function 𝑿(sys::MPS, i::Int64)
    __apply_single_qb_gate(sys, i, [0 1;1 0])
end

function 𝒀(sys::MPS, i::Int64)
    __apply_single_qb_gate(sys, i, [0 -im;im 0])
end

function 𝒁(sys::MPS, i::Int64)
    __apply_single_qb_gate(sys, i, [1 0;0 -1])
end

function 𝑯(sys::MPS, i::Int64)
    __apply_single_qb_gate(sys, i, 1/sqrt(2)*[1 1;1 -1])
end

function 𝑺(sys::MPS, i::Int64)
    __apply_single_qb_gate(sys, i, [1 0;0 im])
end

function 𝑻(sys::MPS, i::Int64)
    __apply_single_qb_gate(sys, i, [1 0;0 exp(im*pi/4)])
end

import LinearAlgebra:svd, Diagonal

function tsvd(a::AbstractArray{𝕋, ℕ}, axes::Tuple) where {𝕋, ℕ}
    a = Number.(a)
    sizeA = size(a)
    laxes = filter(x -> !(x in axes), collect(1:ℕ)) # not in the axes, permute to left side
    raxes = [axes...] # in the axes, permute to right side
    
    lsize = [sizeA[x] for x in laxes] # store original size for reshaping after the svd is done
    rsize = [sizeA[x] for x in raxes]
    
    # permute A, reshape to matrix then do svd
    u, s, vt = svd(reshape(permutedims(a, [laxes;raxes]), prod(lsize), prod(rsize)))
    u = u * sqrt(Diagonal(s)) # s will be absorbed into u and vt
    vt = vt * sqrt(Diagonal(s))
    # NOTES: I have spend alot of time on this line of code
    v = transpose(vt)
    
    # reshape to original order
    dim = length(s)
    return (reshape(u, lsize..., dim), reshape(v, dim, rsize...))
end

# get one controlled single-qubit gate
# note that this gate can only be applied on two adjacent site
function __controlled_gate(gate::AbstractArray)
    size(gate) !== (2,2) && error("parameter wrong")
    # generate |0><0|⨂Identity
    a = contract([1 0;0 0], [1 0;0 1], (), ())
    # generate |1><1|⨂OnebodyGate
    b = contract([0 0;0 1], gate, (), ())
    return permutedims(a+b, [1,3,2,4])
end

# get one controlled single-qubit gate
# note that this gate can only be applied on two adjacent site
function __reverse_gate(gate::AbstractArray)
    size(gate) != (2,2,2,2) && error("parameter wrong")
    return permutedims(gate, [2,1,4,3])
end

# get one swap gate
# note that this gate can only be applied on two adjacent site
function __swap_gate()
    cnot₁ = __controlled_gate([0 1;1 0])
    cnot₂ = deepcopy(cnot₁)
    cnot₃ = deepcopy(cnot₁)
    g₂ = contract(cnot₁, cnot₂, (3,4), (2,1))
    return contract(g₂, cnot₃, (4,3), (1,2))
end

function __adjacent_CNOT(sys::MPS, con::Int64, op::Int64)
    abs(op - con) != 1 && error("parameter wrong")
    g₂ = __controlled_gate([0 1;1 0])
    con > op && begin con, op = op, con; g₂ = __reverse_gate(g₂) end
    sites = contract(sys.data[con], sys.data[op], (3,), (1,))
    sites = contract(sites, g₂, (2,3), (3,4))
    u, vt = tsvd(sites, (4,2))
    sys.data[con] = u
    sys.data[op] = vt
    return (u, vt)
end

function __adjacent_SWAP(sys::MPS, i₁::Int64, i₂::Int64)
    abs(i₂ - i₁) != 1 && error("parameter wrong")
    i₁ > i₂ && begin i₁, i₂ = i₂, i₁ end
    g₂ = __swap_gate()
    sites = contract(sys.data[i₁], sys.data[i₂], (3,), (1,))
    sites = contract(sites, g₂, (2,3), (3,4))
    u, vt = tsvd(sites, (4,2))
    sys.data[i₁] = u
    sys.data[i₂] = vt
    return (u, vt)
end

function CNOT(sys::MPS, con::Int64, op::Int64)
    g₂ = __controlled_gate([0 1;1 0])
    con > op && begin con, op = op, con; g₂ = __reverse_gate(g₂) end
    sorder = [(x, x+1) for x in con:op-2]
    revorder = reverse(sorder)
    for (x, y) in sorder
        __adjacent_SWAP(sys, x, y)
    end
    sites = contract(sys.data[op-1], sys.data[op], (3,), (1,))
    sites = contract(sites, g₂, (2,3), (3,4))
    u, vt = tsvd(sites, (4,2))
    sys.data[op-1] = u
    sys.data[op] = vt
    for (x, y) in revorder
        __adjacent_SWAP(sys, x, y)
    end
end

function CZ(sys::MPS, con::Int64, op::Int64)
    g₂ = __controlled_gate([1 0;0 -1])
    con > op && begin con, op = op, con; g₂ = __reverse_gate(g₂) end
    sorder = [(x, x+1) for x in con:op-2]
    revorder = reverse(sorder)
    for (x, y) in sorder
        __adjacent_SWAP(sys, x, y)
    end
    sites = contract(sys.data[op-1], sys.data[op], (3,), (1,))
    sites = contract(sites, g₂, (2,3), (3,4))
    u, vt = tsvd(sites, (4,2))
    sys.data[op-1] = u
    sys.data[op] = vt
    for (x, y) in revorder
        __adjacent_SWAP(sys, x, y)
    end
end

# discard₀ not using yet
function projection(sys::MPS, basis::AbstractArray, output::Bool=false, discard₀=false)
    length(basis) != length(sys.data) && error("parameter wrong, projection and sys's dims not equal")
    proj = initmps(basis)
    nqubits = length(sys.data)
    t = contract(sys.data[nqubits], proj.data[nqubits], (2,), (2,))
    for x in (nqubits-1):-1:1
        t₀ = contract(sys.data[x], proj.data[x], (2,), (2,))
        t = contract(t, t₀, (1,3), (2,4))
        # these three methods should be contract to one method, but i don't know how to yet
        t = permutedims(t, (2,1,3,4))
        t = permutedims(t, (1,2,4,3))
        t = permutedims(t, (4,2,3,1))
    end
    res = reshape(t, 1)[1]
    if output
        
        println("projection for |ϕ>=|", join(basis, ""), "> == ", res)
    end
    return res
end

function projection_all(sys::MPS, message::String="", discard₀=false)
    println(message)
    nqubits = length(sys.data)
    for x in 0:(2^nqubits - 1)
        n = reverse(digits(x, base=2))
        basis = [[0 for _ in 1:(nqubits-length(n))];n]
        projection(sys, basis, true, discard₀)
    end
end

function measure(sys::MPS, i::Int64)
    meas = deepcopy(sys)
    meas.data[i] = permutedims(contract(meas.data[i], [1 0;0 0], (2,), (2,)), [1,3,2])
    nqubits = length(meas.data)
    t = contract(meas.data[nqubits], sys.data[nqubits], (2,), (2,))
    for x in (nqubits-1):-1:1
        t₀ = contract(meas.data[x], sys.data[x], (2,), (2,))
        t = contract(t, t₀, (1,3), (2,4))
        # these three methods should be contract to one method, but i don't know how to yet
        t = permutedims(t, (2,1,3,4))
        t = permutedims(t, (1,2,4,3))
        t = permutedims(t, (4,2,3,1))
    end
    if rand() < reshape(t, 1)[1]
        return 0
    else
        return 1
    end
end
function measure2(sys::MPS, i::Int64)
    meas = deepcopy(sys)
    meas.data[i] = permutedims(contract(meas.data[i], [1 0;0 0], (2,), (2,)), [1,3,2])
    nqubits = length(meas.data)
    t = contract(meas.data[nqubits], sys.data[nqubits], (2,), (2,))
    for x in (nqubits-1):-1:1
        t₀ = contract(meas.data[x], sys.data[x], (2,), (2,))
        t = contract(t, t₀, (1,3), (2,4))
        # these three methods should be contract to one method, but i don't know how to yet
        t = permutedims(t, (2,1,3,4))
        t = permutedims(t, (1,2,4,3))
        t = permutedims(t, (4,2,3,1))
    end
    return reshape(t, 1)[1]
    if rand() < reshape(t, 1)[1]
        return 0
    else
        return 1
    end
end

measure2 (generic function with 1 method)

In [24]:
#=
    This section test for __adjacent... gate and Bell State
=#

function test2qu(sys::MPS, basis::AbstractArray)
    projection(sys, basis)
end

function test(sys::MPS, message::String="")
    println(message)
    println("|00> : ", test2qu(sys, [0,0]))
    println("|01> : ", test2qu(sys, [0,1]))
    println("|10> : ", test2qu(sys, [1,0]))
    println("|11> : ", test2qu(sys, [1,1]))
end

mps = initmps([0,0])
test(mps, "init")
𝑯(mps, 1)
test(mps, "after 𝑯₁")
__adjacent_CNOT(mps, 2, 1)
test(mps, "after 𝑯₁ and CNOT₂₋₁")
__adjacent_CNOT(mps, 1, 2)
test(mps, "after 𝑯₁, CNOT₂₋₁ and CNOT₁₋₂")
__adjacent_SWAP(mps, 1, 2)
test(mps, "after 𝑯₁, 2 * CNOT₁₋₂ and SWAP₁₋₂")
__adjacent_SWAP(mps, 2, 1)
test(mps, "after 𝑯₁, 2 * CNOT₁₋₂, SWAP₁₋₂ and SWAP₂₋₁")

init
|00> : 1
|01> : 0
|10> : 0
|11> : 0
after 𝑯₁
|00> : 0.7071067811865475
|01> : 0.0
|10> : 0.7071067811865475
|11> : 0.0
after 𝑯₁ and CNOT₂₋₁
|00> : 0.7071067811865475
|01> : 0.0
|10> : 0.7071067811865475
|11> : 0.0
after 𝑯₁, CNOT₂₋₁ and CNOT₁₋₂
|00> : 0.7071067811865475
|01> : 0.0
|10> : 0.0
|11> : 0.7071067811865475
after 𝑯₁, 2 * CNOT₁₋₂ and SWAP₁₋₂
|00> : 0.7071067811865475
|01> : 0.0
|10> : 0.0
|11> : 0.7071067811865475
after 𝑯₁, 2 * CNOT₁₋₂, SWAP₁₋₂ and SWAP₂₋₁
|00> : 0.7071067811865475
|01> : 0.0
|10> : 0.0
|11> : 0.7071067811865475


In [28]:
#=
    This section test for general quantum gate
=#
mps = initmps([0,0,0,0])
𝑯(mps, 1)
CNOT(mps, 1, 4)
projection_all(mps)
CZ(mps, 1, 2)
projection_all(mps)


projection for |ϕ>=|0000> == 0.7071067811865471
projection for |ϕ>=|0001> == 0.0
projection for |ϕ>=|0010> == 0.0
projection for |ϕ>=|0011> == 0.0
projection for |ϕ>=|0100> == 0.0
projection for |ϕ>=|0101> == 0.0
projection for |ϕ>=|0110> == 0.0
projection for |ϕ>=|0111> == 0.0
projection for |ϕ>=|1000> == 0.0
projection for |ϕ>=|1001> == 0.7071067811865471
projection for |ϕ>=|1010> == 0.0
projection for |ϕ>=|1011> == 0.0
projection for |ϕ>=|1100> == 0.0
projection for |ϕ>=|1101> == 0.0
projection for |ϕ>=|1110> == 0.0
projection for |ϕ>=|1111> == 0.0

projection for |ϕ>=|0000> == 0.7071067811865471
projection for |ϕ>=|0001> == 0.0
projection for |ϕ>=|0010> == 0.0
projection for |ϕ>=|0011> == 0.0
projection for |ϕ>=|0100> == 0.0
projection for |ϕ>=|0101> == 0.0
projection for |ϕ>=|0110> == 0.0
projection for |ϕ>=|0111> == 0.0
projection for |ϕ>=|1000> == 0.0
projection for |ϕ>=|1001> == 0.7071067811865471
projection for |ϕ>=|1010> == 0.0
projection for |ϕ>=|1011> == 0.0
projection for

In [32]:
#=
    This section test for quantum teleportation
=#
# prepare |ϕ> and EPR state
qtele = initmps([undef,0,0])
𝑯(qtele, 2)
CNOT(qtele, 2, 3)

# quantum teleportation
CNOT(qtele, 1, 2)
𝑯(qtele, 1)
M₁ = measure(qtele, 1)
M₂ = measure(qtele, 2)
M₂ == 1 && 𝑿(qtele, 3)
M₁ == 1 && 𝒁(qtele, 3)

projection_all(qtele)

# get the result
println(qtele.data[1])
println(qtele.data[3])


projection for |ϕ>=|000> == 0.21440240020370352
projection for |ϕ>=|001> == 0.45169858399920926
projection for |ϕ>=|010> == 0.45169858399920926
projection for |ϕ>=|011> == 0.21440240020370352
projection for |ϕ>=|100> == -0.21440240020370352
projection for |ϕ>=|101> == 0.45169858399920926
projection for |ϕ>=|110> == 0.45169858399920926
projection for |ϕ>=|111> == -0.21440240020370352
Number[0.7329141627288382 0.7329141627288382]

Number[0.5049444125801424 -0.5049444125801424]
Number[0.0 0.8408964152537145; 0.8408964152537145 0.0]


##### PROBLEM
1. contract返回的参数eltype不知道为什么会变成Any，因此在执行tsvd之前必须要用Number进行转换，如tsvd中第一步所示
2. tsvd在完成后，一定要将vt转换成v，这样的u和v才是SVD分解后得到的正确的数据（该问题我卡了至少10个小时）
3. 在contrac函数完成后，没有仔细的进行测试，在最后的乘法之前的reshape，用的是a和b，而不是am和bm